# Similar objects in simbad, GEV and TEV catalogs
First, we import all the packages that we will need.

In [1]:
import warnings
import pandas as pd
import numpy as np
from getsimbad import get_simbad_data
from gevtev import compare_gev_tev_data

warnings.filterwarnings("ignore")

This constant determines a threshold for the similarity between the objects:
$$|glat_{1} - glat_{2}| <= \epsilon$$
$$\epsilon = 1,2*10^{-1}$$

In [2]:
_epsilon = 1.2e-1

The cell above loads data from SIMBAD catalog obtained by requests 'Be', 'O', 'B' and 'plsr'. If the file "data/symbad.txt" does not exist, run getsimbad.py.

In [3]:
try:
    pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', encoding='utf-8')
    del pd_simbad['Unnamed: 0']
except FileNotFoundError:
    pd_simbad = get_simbad_data()


In [4]:
pd_simbad.head()

,s_MAIN_ID,s_RA,s_DEC,s_RA_PREC,s_DEC_PREC,s_COO_ERR_MAJA,s_COO_ERR_MINA,s_COO_ERR_ANGLE,s_COO_QUAL,s_COO_WAVELENGTH,...,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_class,s_glat,s_glon,s_spectra,s_nan_spectra,s_zero_spectra
0,b'HD 201433A',21 08 38.894,+30 12 20.27,7.0,7.0,26.0000,26.0000,90.0,B,O,...,NaN,NaN,NaN,NaN,plsr,-11.774311,76.107743,False,False,False
1,b'* nu. For',02 04 29.4462,-29 17 48.491,14.0,14.0,0.1834,0.1942,90.0,A,O,...,NaN,NaN,NaN,NaN,plsr,-73.777993,224.887601,False,False,False
2,b'* phi Her',16 08 46.1668,+44 56 05.732,14.0,14.0,0.3710,0.3754,90.0,A,O,...,NaN,NaN,NaN,NaN,plsr,47.107400,70.847825,False,False,False
3,b'V* CG And',00 00 43.6345,+45 15 12.002,14.0,14.0,0.0550,0.0422,90.0,A,O,...,NaN,NaN,NaN,NaN,plsr,-16.702455,113.649358,False,False,False
4,b'HD 133029',15 00 38.7184,+47 16 38.793,14.0,14.0,0.0472,0.0421,90.0,A,O,...,NaN,NaN,NaN,NaN,plsr,57.710074,80.191942,False,False,False


In [5]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

In [6]:
pd_simbad['s_nan_spectra'] = pd_simbad[s_spectrum_columns].isna().all(axis=1)

In [7]:
pd_simbad['s_zero_spectra'] = ~(pd_simbad[s_spectrum_columns] != 0).any(axis = 1)

We load data common for GEV and TEV catalogs.

In [8]:
common_data, only_tev_data, only_gev_data = compare_gev_tev_data(_epsilon)

The following cells compare distances from a common catalog of GEV and TEV and SIMBAD catalog by translating SIMBAD coordinates to glan and glot due to astropy module. If their relative distance is smaller than $\epsilon$ they are associated to each another.

In [9]:
def create_vectors_common_gevtev_simbad_old(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: two vectors where each index of TEV(GEV) is associated with its indev in GEV(TEV).
    If such objects does not existe, the corresponding vector coordinate is equal to -1
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    # в данной реализации:
    # если для объекта i gevtev по расстоянию подходит несколько объектов из simbad, 
    # то привязан он будет к последнему
    for i in range(len(glat_gevtev)):
        for j in range(len(glat_simbad)):
            if ((np.abs(glat_gevtev[i] - glat_simbad[j]) < epsilon) 
                and (np.abs(glon_gevtev[i] - glon_simbad[j]) < epsilon)) :
                    C_associations_gevtev[i] = j
                    C_associations_simbad[j] = i
    return C_associations_gevtev, C_associations_simbad

In [10]:
def create_matrix_common_gevtev_simbad(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix with association: m[i][j] = True if object i from gevtev
            is close enough to object j from simbad
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    glat_dif_matrix = np.dot(np.vstack((glat_gevtev, -np.ones_like(glat_gevtev))).T,
                            np.vstack((np.ones_like(glat_simbad), glat_simbad)))
    glon_dif_matrix = np.dot(np.vstack((glon_gevtev, -np.ones_like(glon_gevtev))).T,
                            np.vstack((np.ones_like(glon_simbad), glon_simbad)))
    pairs_matrix = np.logical_and(np.abs(glat_dif_matrix) < epsilon, 
                                np.abs(glon_dif_matrix) < epsilon)
    return pairs_matrix

In [11]:
def create_simbad_pandas_for_gevtev(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a pandas with associating IDs and classes from simbad in the gevtev order
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    glat_dif_matrix = np.dot(np.vstack((glat_gevtev, -np.ones_like(glat_gevtev))).T,
                            np.vstack((np.ones_like(glat_simbad), glat_simbad)))
    glon_dif_matrix = np.dot(np.vstack((glon_gevtev, -np.ones_like(glon_gevtev))).T,
                            np.vstack((np.ones_like(glon_simbad), glon_simbad)))
    pairs_matrix = np.logical_and(np.abs(glat_dif_matrix) < epsilon, 
                                np.abs(glon_dif_matrix) < epsilon)

    
    s_classes_list_of_lists = []
    s_main_id_list_of_lists = []
    s_simbad = [] # 0 - not in simbad, 2 - in simbad without spectra, 1 - in simbad with spectra
    for i in pairs_matrix:
        association_simbad_numbers = np.where(i)
        s_classes_pandas = simbad.iloc[association_simbad_numbers]['s_class']
        s_main_id_pandas = simbad.iloc[association_simbad_numbers]['s_MAIN_ID']
        if len(s_classes_pandas.unique()) == 1:
            s_classes_list = s_classes_pandas.unique()
            s_main_id_list = s_main_id_pandas.values
        else:
            s_classes_list = s_classes_pandas.values
            s_main_id_list = s_main_id_pandas.values
        s_classes_list_of_lists.append(s_classes_list)
        s_main_id_list_of_lists.append(s_main_id_list)
        if len(s_classes_list) == 0:
            s_simbad.append(0)
        else:
            if simbad.iloc[association_simbad_numbers]['s_nan_spectra'].any():
                s_simbad.append(2)
            else:
                s_simbad.append(1)
        
    simbad_no_spectra_columns = ['s_MAIN_ID', 's_class', 's_simbad'] 
    simbad_no_spectra_pandas = pd.DataFrame(columns = simbad_no_spectra_columns)   
    simbad_no_spectra_pandas['s_MAIN_ID'] = s_main_id_list_of_lists
    simbad_no_spectra_pandas['s_class'] = s_classes_list_of_lists
    simbad_no_spectra_pandas['s_simbad'] = s_simbad
    return simbad_no_spectra_pandas

In [12]:
def create_common_data(data_gevtev, data_simbad, pairs_matrix, catalog="tev"):
    """The fonction adds objects found both in GeV, TeV and simbad.
    """
    vector_association = np.where(np.sum(pairs_matrix, axis=0) > 0)[0]
    pd_common_gevtev = pd.DataFrame()
    for i in vector_association:
        data_gevtev_join = (pairs_matrix[:, i] > 0)*(i + 1) - 1
        data_gevtev["join"] = data_gevtev_join
        pd_common_gevtev0 = pd.merge(data_gevtev, data_simbad, right_index=True, left_on='join', how='inner')
        if (len(pd_common_gevtev)):
            pd_common_gevtev = pd_common_gevtev.append(pd_common_gevtev0)
            del pd_common_gevtev["join"]
            del data_gevtev["join"]
        else:
            pd_common_gevtev = pd_common_gevtev0.copy()
    
    
    if (catalog.find("tev") != -1):    
        array_non_duplicate = [catalog+'_glat', catalog+'_glon', 's_glat','s_glon']
    else:
        array_non_duplicate = [catalog+'_GLAT', catalog+'_GLON', 's_glat','s_glon']
    pd_common_gevtev = pd_common_gevtev.drop_duplicates(array_non_duplicate)
    pd_common_gevtev = pd_common_gevtev.reset_index()

    #df_common = pd.DataFrame(data = data, columns = namefinal)
    return pd_common_gevtev

In [13]:
matrix = create_matrix_common_gevtev_simbad(common_data, pd_simbad, _epsilon)
pd_common_gevtevsimbad = create_common_data(common_data, pd_simbad, matrix)
print("There are " + str(len(pd_common_gevtevsimbad)) + " common objects in the three catalogs.")

There are 68 common objects in the three catalogs.


In [14]:
pd_simbad_for_gevtev = create_simbad_pandas_for_gevtev(common_data, pd_simbad, _epsilon)
pd_common_gevtev = pd.merge(common_data, pd_simbad_for_gevtev, how='inner', left_index=True, right_index=True)

All gevtev objects with simbad columns containing in s_simbad :
- 0 - if near this gevtev object any simbad object is found
- 1 - if near this gevtev object a simbad object is found and its spectrum is given
- 2 - if near this gevtev object a simbad object is found and its spectrum is not given

in s_MAIN_ID:

- list of simbad names in corresponding objects

in s_class:

If all corresponding objects have the same class - its name
if not - classes in the same order as names in s_MAIN_ID

In [15]:
pd_common_gevtev

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
0,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'pl',b'70hRXZ',213,b'TeV J0013-188',2,b'TeV J0013-1853',b'egal',[],[],0
1,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'pl',b'xCyAjF',209,b'TeV J0025+641',3,b'TeV J0025+6410',b'gal',[],[],0
2,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'none',b'PKJZqs',239,b'TeV J0033-193',4,b'TeV J0033-1921',b'egal',[],[],0
3,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'none',b'eNYfrm',229,b'TeV J0035+598',5,b'TeV J0035+5947',b'egal',[],[],0
4,193,b'0FGL J0136.6+3903',b'1FGL J0136.5+3905 ',b'1FHL J0136.5+3906 ',b'2FGL J0136.5+3905 ',b'B3 0133+388 ',b' ',b' ',b' ',b' ',...,b'none',b'7WhQrw',244,b'TeV J0136+391',7,b'TeV J0136+3905',b'egal',[],[],0
5,214,b' ',b' ',b'1FHL J0152.6+0148 ',b'2FGL J0152.6+0148 ',b'PMN J0152+0146 ',b' ',b' ',b' ',b' ',...,b'pl',b'tKGVcC',156,b'TeV J0152+017',8,b'TeV J0152+0146',b'egal',[],[],0
6,241,b' ',b'1FGL J0205.6+6449 ',b'1FHL J0205.7+6448 ',b'2FGL J0205.8+6448 ',b'PSR J0205+6449 ',b' ',b' ',b' ',b' ',...,b'pl',b'2I2TuY',252,b'TeV J0209+648',9,b'TeV J0205+6451',b'gal',[],[],0
7,273,b'0FGL J0220.9+3607',b'1FGL J0221.0+3555 ',b'1FHL J0221.1+3555 ',b'2FGL J0221.0+3555 ',b'B0218+357 ',b' ',b' ',b' ',b' ',...,b'none',b'hJnA37',254,b'TeV J0218+359',10,b'TeV J0221+3556',b'egal',[],[],0
8,278,b'0FGL J0222.6+4302',b'1FGL J0222.6+4302 ',b'1FHL J0222.6+4302 ',b'2FGL J0222.6+4302 ',b'3C 66A ',b' ',b' ',b'3EG J0222+4253',b'EGR J0223+4300 ',...,b'none',b'J01xzR',119,b'TeV J0222+430',11,b'TeV J0222+4302',b'egal',[],[],0
9,278,b'0FGL J0222.6+4302',b'1FGL J0222.6+4302 ',b'1FHL J0222.6+4302 ',b'2FGL J0222.6+4302 ',b'3C 66A ',b' ',b' ',b'3EG J0222+4253',b'EGR J0223+4300 ',...,b'none',b'y6gVxa',138,b'TeV J0223+430',12,b'TeV J0223+4259',b'unid',[],[],0


ALL gevtev objects found in simbad

In [16]:
pd_common_gevtev[pd_common_gevtev['s_simbad']>0]

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
11,319,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,b'pl',b'fW749W',89,b'TeV J0240+612',14,b'TeV J0240+6115',b'gal',"[b'LS I +61 303', b'LS I +61 303']",[B0Ve],1
35,1119,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',...,b'none',b'wN7SNs',194,b'TeV J0955+696',40,b'TeV J0955+6940',b'egal',"[b'[CHP2004] J095549.8+693934', b'[MTK2001b] 8...",[B0Ve],2
40,1172,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,b'pl',b'5FlzLA',237,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',[b'2FGL J1019.0-5856'],[B0Ve],2
83,2222,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',"[b'SSTGC 519103', b'SSTGC 519103', b'GCIRS 1W...","[Be, B, B, O]",1
84,2226,b' ',b'1FGL J1745.6-2900c',b'1FHL J1745.6-2900 ',b'2FGL J1745.6-2858 ',b' ',b' ',b' ',b' ',b' ',...,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',"[b'SSTGC 519103', b'SSTGC 519103', b'GCIRS 1W...","[Be, B, B, O]",1
85,2228,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'none',b'LsZtAg',269,b'TeV J1746-289',-9223372036854775808,b'',b'gal',[b'[MCD2010] 13'],[O],1
86,2228,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'pl',b'LsZtAg',277,b'TeV J1746-289',-9223372036854775808,b'',b'gal',"[b'[MCD2010] 18', b'[MCD2010] 13']",[O],1
90,2358,b'0FGL J1826.3-1451',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',...,b'ecpl',b'xKaTtF',96,b'TeV J1826-148',119,b'TeV J1826-1449',b'gal',[b'V* V479 Sct'],[B0Ve],1
92,2405,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,b'pl',b'amNIwH',142,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal',"[b'LS IV -05 11', b'[KRL2007b] 332']","[B, B0Ve]",1
103,2648,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,b'none',b'1FCN0q',149,b'TeV J2031+406',145,b'TeV J2031+4040',b'gal',"[b'[MT91] 213', b'Schulte 21']",[B0Ve],1


In [17]:
pd_common_gevtevsimbad.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_pl_norm_err,tev_spec_pl_norm_err_sys,tev_spec_theta,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,1.300000e-12,2.000000e-13,0.70,b'pl',b'amNIwH',142,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal'
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,5.000000e-14,NaN,0.09,b'pl',b'LsZtAg',277,b'TeV J1746-289',-9223372036854775808,b'',b'gal'
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'


The next cell chooses several columns from a dataFrame to make sure that the output is correct(objects have the same coordinates etc.)

In [18]:
columns_to_compare = ['s_glat',
                     's_glon',
                     'gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     's_class',
                     'gev_CLASS1',
                     'tev_classes']
pd_common_gevtevsimbad[columns_to_compare].head()

,s_glat,s_glon,gev_ASSOC_TEV,tev_glat,gev_GLAT,tev_glon,gev_GLON,s_class,gev_CLASS1,tev_classes
0,-0.011453,359.985525,b' ',-0.046162,-0.020071,359.944244,359.863953,Be,b' ',b'unid'
1,-0.198384,26.857782,b'HESS J1841-055 ',-0.197604,-0.198590,26.795113,26.795933,B,b'PWN ',b'unid'
2,-0.047327,359.945163,b' ',-0.046162,-0.020071,359.944244,359.863953,B,b' ',b'unid'
3,-0.070598,0.237663,b' ',-0.113206,-0.103157,0.140617,0.149201,O,b'pwn ',b'unid'
4,-0.042219,359.951421,b' ',-0.046162,-0.020071,359.944244,359.863953,O,b' ',b'unid'


In [19]:
columns_to_compare_gevtev = ['gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     'gev_CLASS1',
                     'tev_classes']

In [20]:
common_data.to_csv("gevtev.txt", sep='\t')

The next cells find common values in simbad-tev

In [21]:
matrix = create_matrix_common_gevtev_simbad(only_tev_data, pd_simbad, _epsilon)
pd_common_onlytevsimbad = create_common_data(only_tev_data, pd_simbad, matrix)
print("There are " + str(len(pd_common_onlytevsimbad)) + " common objects in tev-simbad catalogs.")

There are 123 common objects in tev-simbad catalogs.


The next cells find common values in simbad-gev

In [22]:
matrix = create_matrix_common_gevtev_simbad(only_gev_data, pd_simbad, _epsilon, catalog="gev")
pd_common_onlygevsimbad = create_common_data(only_gev_data, pd_simbad, matrix, catalog="gev")
print("There are " + str(len(pd_common_onlygevsimbad)) + " common objects in gev-simbad catalogs.")

There are 155 common objects in gev-simbad catalogs.


In [23]:
pd_common_gevtevsimbad.to_csv("data/gevtevsimbad.txt", sep='\t')
pd_common_onlygevsimbad.to_csv("data/gevsimbad.txt", sep='\t')
pd_common_onlytevsimbad.to_csv("data/tevsimbad.txt", sep='\t')
common_data.to_csv("data/gevtev.txt", sep='\t')

In [24]:
only_gev_data.to_csv("data/gev.txt", sep='\t')
only_tev_data.to_csv("data/tev.txt", sep='\t')
pd_simbad.to_csv("data/simbad.txt", sep='\t')

In [25]:
pd_common_gevtev.to_csv("data/gevtev_simbadclasses.txt", sep='\t')

In [26]:
print(str(np.sum(pd_simbad['s_nan_spectra'])) + ' simbad objects do not have simbad spectra. In total '+ str(len(pd_simbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_simbad['s_zero_spectra'])) + ' simbad objects have zero simbad spectra. In total '+ str(len(pd_simbad['s_zero_spectra'] )) + ' objects.')
print(str(np.sum(pd_common_gevtevsimbad['s_nan_spectra'])) + ' gevtevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_gevtevsimbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_common_onlygevsimbad['s_nan_spectra'])) + ' gevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_onlygevsimbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_common_onlytevsimbad['s_nan_spectra'])) + ' tevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_onlytevsimbad['s_nan_spectra'])) + ' objects.')

1308 simbad objects do not have simbad spectra. In total 11625 objects.
0 simbad objects have zero simbad spectra. In total 11625 objects.
54 gevtevsimbad objects do not have simbad spectra. In total 68 objects.
63 gevsimbad objects do not have simbad spectra. In total 155 objects.
56 tevsimbad objects do not have simbad spectra. In total 123 objects.


In [27]:
no_spectra_gevtevsimbad_objects = pd_common_gevtevsimbad[pd_common_gevtevsimbad['s_nan_spectra']]['s_MAIN_ID'].unique()

In [28]:
no_spectra_gevtevsimbad_objects.sort()
for i in no_spectra_gevtevsimbad_objects:
    print(i)

b'2FGL J1019.0-5856'
b'CXOM82 J095550.5+694022'
b'CXOU J095510.7+693955'
b'CXOU J095527.1+693923'
b'CXOU J095527.2+694050'
b'CXOU J095531.8+693957'
b'CXOU J095534.5+693823'
b'CXOU J095544.8+694100'
b'CXOU J095546.1+694026'
b'CXOU J095546.5+694040'
b'CXOU J095547.4+694059'
b'CXOU J095547.5+694120'
b'CXOU J095548.9+694010'
b'CXOU J095549.1+694036'
b'CXOU J095550.1+694034'
b'CXOU J095550.2+694039'
b'CXOU J095550.5+694056'
b'CXOU J095550.6+694043'
b'CXOU J095550.7+694039'
b'CXOU J095550.9+694044'
b'CXOU J095551.0+694050'
b'CXOU J095551.2+694012'
b'CXOU J095551.3+694043'
b'CXOU J095551.4+694035'
b'CXOU J095552.2+694042'
b'CXOU J095552.4+694024'
b'CXOU J095552.7+694045'
b'CXOU J095552.8+694020'
b'CXOU J095553.0+694048'
b'CXOU J095553.2+694052'
b'CXOU J095553.3+694101'
b'CXOU J095553.9+694045'
b'CXOU J095554.1+694039'
b'CXOU J095554.9+694038'
b'CXOU J095555.1+694027'
b'CXOU J095600.7+694107'
b'CXOU J095601.7+694308'
b'CXOU J095608.0+694139'
b'CXOU J095614.7+694249'
b'CXOU J095615.1+693951'
b'